In [51]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import signal
from scipy import optimize
import os
import time as timemodule
import sys
from IPython.display import clear_output

In [2]:
data = pd.read_excel('./data/laundry_from_drive_manual_merge.xlsx')
data=data.dropna(subset=['Machines free'])
data=data.dropna(subset=['Dryers free'])
data=data.dropna(subset=['Rain'])
data=data.dropna(subset=['Temperature'])

In [59]:
timing = [timemodule.perf_counter()]
day_list = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
time = np.arange(0, 24*60-1, 1).astype(np.int)
temperature = np.arange(int(np.min(data["Temperature"])),int(np.max(data["Temperature"])),1)
rain = np.arange(int(np.min(data["Rain"])),int(np.max(data["Rain"])),10)

time_from_data = (data["Hour"]*60+data["Minute"]).astype(np.int)

washing_predicted = np.zeros(shape = (len(day_list), time.size, temperature.size, rain.size))
dryers_predicted = np.zeros(shape = (len(day_list), time.size, temperature.size, rain.size))

empty = 0

for i, d in enumerate(day_list):
    mask_d = data["Day of the week"] == d
    for j, t in enumerate(time):
        if j % 25 == 0:
            timing.append(timemodule.perf_counter())
            time_est = float(timing[-1]-timing[0])/((24*60*i+t)/(60*24*7))
            try:
                clear_output(wait=True)
                print("%0.1f%% done.\nTime elapsed: %d minutes %d seconds.\nEstimated total time: %d minutes %d seconds"%(
                             (24*60*i+t)/(60*24*7)*100, 
                             (timing[-1]-timing[0])/60,
                             (timing[-1]-timing[0])%60,
                             time_est/60, 
                             time_est%60
                          ))
            except:
                print("%0.1f%% done"%((24*60*i+t)/(60*24*7)*100))
        mask_t = time_from_data == t
        for k, T in enumerate(temperature):
            mask_T = data["Temperature"] == T
            for l, p in enumerate(rain):
                mask_p = data["Rain"] == p
                washing_predicted[i,j,k,l] = np.nanmean(data["Machines free"][mask_d*mask_t*mask_T*mask_p])
                dryers_predicted[i,j,k,l] = np.nanmean(data["Dryers free"][mask_d*mask_t*mask_T*mask_p])

print("Done! Saving results...")
# Omg save it to disk please
np.save("data/washing predicted.npy", washing_predicted)
np.save("data/drying predicted.npy", dryers_predicted)
np.save("temp and rain.npy",(temperature, rain))
print("Saved! Some stats:")
print("Saved calculating %d loops"%empty)
print("There are %d valid datapoints in the set"%np.count_nonzero(~(np.isnan(washing_predicted)*np.isnan(dryers_predicted))))
print("My poorly written code executed in %02d minutes %02d seconds. Good thing I just saved it!"%((timemodule.perf_counter()-timing[0])/60, 
                                                                      (timemodule.perf_counter()-timing[0])%60))

99.9% done.
Time elapsed: 34 minutes 20 seconds.
Estimated total time: 34 minutes 23 seconds
Done! Saving results...


NameError: name 'drying_predicted' is not defined

## If you want to manually save, you can below.

In [60]:
np.save("data/washing predicted.npy", washing_predicted)
np.save("data/drying predicted.npy", dryers_predicted)
np.save("temp and rain.npy",(temperature, rain))